In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
df = pd.read_csv("data/migr_imm3ctb.tsv", sep="\t|,")
df

/Users/jonathanbleiberg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
def process_df(df):
    df.rename(columns={"geo\\time":"c_dest"}, inplace=True)
    df = df[df.age == "TOTAL"]
    df = df[df.sex == "T"]
    df = df[df.agedef == 'COMPLET']
    df = df.sort_values("c_dest")
    df.drop(["age", "agedef", "unit", "sex"], axis=1, inplace=True)
    df = df[['c_dest','c_birth','2017 ','2016 ','2015 ','2014 ','2013 ',
             '2012 ','2011 ','2010 ','2009 ','2008']]
    df.rename(columns=lambda x: x.strip(), inplace=True)
    return df

In [5]:
df = process_df(df)
df

,c_dest,c_birth,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
46,AT,AD,0,1,1,0,:,0,0,:,:,1
12673,AT,MM,12,9,19,7,:,11,4,:,:,9
12742,AT,MN,115,191,347,229,:,239,209,:,:,167
12813,AT,MR,6,6,3,1,:,7,3,:,:,2
1904,AT,ASI_W,7653,11295,31782,:,:,2208,1587,1295,:,1581
12885,AT,MS,0,0,0,0,:,0,0,:,:,0
12957,AT,MT,10,13,9,9,:,9,6,:,:,13
13027,AT,MU,6,3,3,8,:,6,8,:,:,12
13096,AT,MV,2,2,1,1,:,0,1,:,:,1
1829,AT,ASI_S_E,1055,975,1065,:,:,870,888,729,:,888


In [6]:
eu_countries = df.c_dest.unique()

# restrict to eu countries
eu_df = df[df.c_birth.isin(eu_countries)].copy()
eu_df.reset_index(drop=True, inplace=True)

In [7]:
# extract one year of migration data from raw df
def extract_year_df(year, raw_df):
    raw_df = raw_df.copy()
    year = str(year)
    raw_df.rename(columns=lambda x: x.strip(), inplace=True)
    raw_df.loc[:, year] = raw_df.loc[:, year].str.strip()
    raw_df.loc[:, year] = raw_df.loc[:, year].str.replace('[a-z]| ', "")
    raw_df.loc[:, year] = raw_df.loc[:, year].replace(':', np.nan)
    raw_df = raw_df.dropna()
    raw_df.loc[:, year] = raw_df.loc[:, year].astype(np.int32)
    raw_df = raw_df.loc[:, ['c_dest','c_birth',year]]
    raw_df = raw_df[raw_df.c_dest != raw_df.c_birth]
    raw_df.columns= ['c_dest','c_birth','weight'] 
    return raw_df

In [8]:
# create network model of migration in given year
def create_network(raw_df, year):
    raw_df = extract_year_df(year, raw_df)
    G = nx.from_pandas_edgelist(raw_df, source='c_birth', target='c_dest', 
                                edge_attr='weight', create_using=nx.DiGraph)
    
    return G

In [9]:
for year in range(2008, 2018):
    G = create_network(eu_df, year)
    print(f"{year}: {G.number_of_nodes()} nodes; {G.number_of_edges()} edges")

2008: 32 nodes; 544 edges
2009: 32 nodes; 550 edges
2010: 32 nodes; 580 edges
2011: 32 nodes; 641 edges
2012: 32 nodes; 674 edges
2013: 32 nodes; 454 edges
2014: 32 nodes; 732 edges
2015: 32 nodes; 703 edges
2016: 32 nodes; 702 edges
2017: 32 nodes; 732 edges


In [19]:
G = create_network(eu_df, 2017)

In [20]:
list(nx.strongly_connected_components(G))

[{'AT',
  'BE',
  'BG',
  'CH',
  'CZ',
  'DK',
  'EE',
  'ES',
  'FI',
  'FR',
  'HR',
  'HU',
  'IS',
  'IT',
  'LI',
  'LT',
  'LU',
  'LV',
  'NL',
  'NO',
  'RO',
  'SE',
  'SI',
  'SK',
  'UK'},
 {'MT'},
 {'PL'},
 {'PT'},
 {'EL'},
 {'CY'},
 {'DE'},
 {'IE'}]

In [26]:
G.in_edges('FR', data=True)

InEdgeDataView([('PT', 'FR', {'weight': 7995}), ('LT', 'FR', {'weight': 308}), ('EL', 'FR', {'weight': 1112}), ('SK', 'FR', {'weight': 414}), ('DE', 'FR', {'weight': 6449}), ('NL', 'FR', {'weight': 1681}), ('AT', 'FR', {'weight': 386}), ('IE', 'FR', {'weight': 653}), ('FI', 'FR', {'weight': 127}), ('EE', 'FR', {'weight': 39}), ('SI', 'FR', {'weight': 134}), ('DK', 'FR', {'weight': 195}), ('LU', 'FR', {'weight': 848}), ('RO', 'FR', {'weight': 8041}), ('ES', 'FR', {'weight': 9976}), ('BG', 'FR', {'weight': 1484}), ('BE', 'FR', {'weight': 7448}), ('SE', 'FR', {'weight': 579}), ('UK', 'FR', {'weight': 10756}), ('MT', 'FR', {'weight': 16}), ('LV', 'FR', {'weight': 116}), ('HR', 'FR', {'weight': 434}), ('IT', 'FR', {'weight': 12466}), ('PL', 'FR', {'weight': 2686}), ('CZ', 'FR', {'weight': 577}), ('CY', 'FR', {'weight': 63}), ('HU', 'FR', {'weight': 623})])

In [28]:
df[df.c_dest =='DE']

,c_dest,c_birth,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
6861,DE,EU27_FOR,:,:,:,:,:,296591,227206,158220,133436 b,328448
5913,DE,EFTA_FOR,4662 be,4581 be,4589,4300,4394,3962,3819,3432,3193 b,6047
6511,DE,EU15_FOR,:,:,:,:,:,83267,61406,:,:,:
5817,DE,EFTA,4662 be,4581 be,4589,4300,4394,3962,3819,3432,3193 b,6047
12036,DE,MDC_EXT,:,:,:,:,:,97788,77709,63068,60828 b,89727
7152,DE,EXT,:,:,:,:,:,206522,168952,145927,139997 b,209178
13873,DE,NEU28_FOR,449934 be,549831 be,999414,416181,284185,:,:,:,:,:
7189,DE,EXT_EU27,:,:,:,:,:,:,:,:,160160 b,:
8865,DE,HDC_EXT,:,:,:,:,:,86185,71096,61625,58666 b,100113
6430,DE,EU15,:,:,:,:,:,144503,119679,:,:,:
